# Almost Visual Inertial Odometry

# Imports

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import MobileNetV2
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from tqdm import tqdm
from  matplotlib import pyplot as plt
from matplotlib.image import imread
import pandas as pd
import numpy as np

# Constants

In [2]:
SEED = 20197
BATCH_SIZE = 16
TEST_BATCH_SIZE = 32
GAMMA = 0.1   # torch default
LR=0.001 
EPOCHS = 50

# Device

In [3]:
available_cuda = torch.cuda.is_available()
print(f"Available cuda: {available_cuda}")

device = torch.device("cuda" if available_cuda else "cpu")

Available cuda: False


# Dataset

In [62]:
class AdvioDataset(torch.utils.data.Dataset):
      def __init__(self, path_frames, path_frames_folder, path_inertials, path_labels):
            self.path_frames_folder = path_frames_folder
            self.path_labels = path_labels
            self.path_inertials = path_inertials
            self.df_frames = pd.read_csv(path_frames)

      def __len__(self):
            return len(self.df_frames) - 1

      def __getitem__(self, index):

            # load sample of frames
            image_name = self.path_frames_folder + self.df_frames.iloc[index, 1]
            frame = imread(image_name)
            frame = torch.Tensor(frame)
            frame = torch.moveaxis(frame, -1, 0)

            image_name_next = self.path_frames_folder + self.df_frames.iloc[index+1, 1]
            frame_next = imread(image_name_next)
            frame_next = torch.Tensor(frame_next)
            frame_next = torch.moveaxis(frame_next, -1, 0)

            sample_frame = frame_next - frame

            # TODO build buffer sample of inertials
            sample_inertials = ""

            # load labels
            label_odometry = np.load(self.path_labels + f"{self.df_frames.iloc[index, 0]}.npy")
            label_odometry = torch.Tensor(label_odometry)

            # load inertials
            label_inertial = np.load(self.path_inertials + f"{self.df_frames.iloc[index, 0]}.npy")
            label_inertial = torch.Tensor(label_inertial)

            return sample_frame, label_odometry, label_inertial


path_frames = "./data/advio-01/iphone/frames_synced.csv"
path_frames_folder = "./data/advio-01/iphone/frames/"
path_inertials = "./data/advio-01/iphone/inertials/"
path_labels = "./data/advio-01/iphone/labels/"

train_data = AdvioDataset(path_frames, path_frames_folder, path_inertials, path_labels)

params = {'batch_size': BATCH_SIZE,
          'shuffle': True,
          'num_workers': 3,
          'drop_last':True}
train_loader = torch.utils.data.DataLoader(train_data, **params)

In [63]:
for idx, (sample, label_odometry, _) in enumerate(train_loader):
    label_odometry = label_odometry.mean(axis=1)
    print(sample.shape)

    break

torch.Size([16, 3, 224, 224])


# Model

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv2d_odometry = MobileNetV2(num_classes=3)

    def forward(self, x):
        x = self.conv2d_odometry(x)
        return x

# Training

In [34]:
def normalize(data):
  data = data - data.min()
  data = data / data.max()
  return data

In [59]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    history_loss = 0

    for batch_idx, (data, labels_odometry, _) in enumerate(train_loader):
        # data preparation
        data = normalize(data)
        labels_odometry = labels_odometry.mean(axis=1)
        data, labels_odometry = data.to(device), labels_odometry.to(device)

        #forward
        optimizer.zero_grad()
        output = model(data)

        # loss
        loss = F.mse_loss(output, label_odometry)

        # backward
        loss.backward()
        optimizer.step()

        #stats
        history_loss += loss.item()

        if batch_idx%100==0:
            print(f"\t[# {batch_idx}] loss: {loss.item()}")

    return history_loss/(len(train_loader)/BATCH_SIZE) # average epoch loss


In [60]:
def test(model, device, test_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(test_loader):
        # data preparation
        data, target = data.to(device), target.to(device)

        # forward
        optimizer.zero_grad()
        output = model(data)

        # loss
        loss = torch.sqrt(F.mse_loss(output, data))

        #stats
        history_loss += loss.item()

    return history_loss/(len(test_loader.dataset)/BATCH_SIZE) # average epoch loss


In [64]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

h_loss_train = []
h_loss_test = []

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")

    loss_train = train(model, device, train_loader, optimizer, epoch)
    #loss_test, data, output, embedded_code = test(model, device, test_loader)

    #populating the history of the loss
    h_loss_train.append(loss_train)
    #h_loss_test.append(loss_test)

Epoch 1/50
	[# 0] loss: 0.04536519572138786
	[# 100] loss: 0.003761247731745243
	[# 200] loss: 0.0021978402510285378
	[# 300] loss: 0.0025940833147615194
	[# 400] loss: 0.0004290762299206108
	[# 500] loss: 0.001128656673245132
	[# 600] loss: 0.0003063115291297436
	[# 700] loss: 0.00010401711915619671
	[# 800] loss: 0.00030234953737817705


AttributeError: 'DataLoader' object has no attribute 'train'

# Plotting